In [ ]:
# Extended code from 
# https://github.com/Azure-Samples/cognitive-services-quickstart-code/blob/master/python/ComputerVision/REST/python-disk.md
# https://hub.gke2.mybinder.org/user/microsoft-cogni-e-vision-python-3ukfjyui/notebooks/Jupyter%20Notebook/Computer%20Vision%20API%20Example.ipynb

# Import the necessary modules
import requests
from PIL import Image
from io import BytesIO
import cv2
import time

# Key for the Computer Vision API from the 'Keys and Endpoint' section in the Azure portal
subscription_key_cv = '<your subscription key>'
# URL Endpoint for the Computer Vision API
endpoint_cv = '<your endpoint>'
# Analyze URL
analyze_url_cv = endpoint_cv + "vision/v3.1/analyze"

# Other variables
interval = 5
count = 1
file_name = 'temp.jpg'

# The display_caption function
def display_caption(imageFile):
    # Read the image into a byte array
    image_data = open(imageFile, "rb").read()
    headers = {'Ocp-Apim-Subscription-Key': subscription_key_cv, 'Content-Type': 'application/octet-stream'}
    params = {'visualFeatures': 'Categories,Description,Tags'}
    response = requests.post(analyze_url_cv, headers=headers, params=params, data=image_data)
    response.raise_for_status()

    analysis = response.json()
    image_caption = analysis["description"]["captions"][0]["text"].capitalize()
    image_tags = analysis["description"]["tags"]
    #print(image_tags)

    img = cv2.imread(imageFile)
    x,y,w,h = 0,25,700,10
    cv2.rectangle(img, (x, x), (x + w, y + h), (0,0,0), -1)
    cv2.putText(img=img, text=image_caption, org=(10, 25), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.7, color=(255, 255, 255),thickness=1)
    cv2.imshow('Processed', img)

# Capture video from file
cap= cv2.VideoCapture('sample.mp4')
# Get the frame rate of the video
fps= int(cap.get(cv2.CAP_PROP_FPS))

while cap.isOpened():
    ret,frame= cap.read()

    if ret == True:

        time.sleep(1/fps)

        cv2.imwrite(file_name, frame)

        cv2.imshow('Original', frame)

        if count == interval:
            display_caption(file_name)
            count = 1
        else:
            count += 1

        # Press Q on keyboard to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        break


cap.release()
cv2.destroyAllWindows()